# Training main:

take from cyanite notebook the interface to observe tags distribution and create datasets


In [18]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn import metrics
from torch.autograd import Variable

from frontend import Frontend_mine, Frontend_won
from backend import Backend, Backend2
from data_loader import get_DataLoader

main_dict = {"frontend_dict":
             {"list_out_channels":[128,128,256,256,256,256], 
              "list_kernel_sizes":[(3,3),(3,3),(3,3),(3,3),(3,3),(3,3)],
              "list_pool_sizes":  [(3,1),(2,2),(2,2),(2,1),(2,1),(2,1)], 
              "list_avgpool_flags":[False,False,False,False,False,True]},
             
             "backend_dict":
             {"n_class":50,
              "bert_config":None, 
              "recurrent_units":2}} # pass None to deactivate

In [19]:
class CRNNSA(nn.Module):
    
    """
    TODO: explore whether "rec_unit -> self_att -> rec_unit"
    would have worked better.
    """
    
    # BERT-based Convolutional Recurrent Neural Network
    # Code adopted from https://github.com/minzwon/sota-music-tagging-models/
    def __init__(self, main_dict=None, backend=None, frontend=None):
        super(CRNNSA, self).__init__()
        

        if main_dict is not None:
            self.frontend = Frontend_mine(main_dict["frontend_dict"])
            self.backend = Backend2(main_dict)
        else:
            self.frontend = frontend
            self.backend = backend


    def forward(self, spec):
        
        x = self.backend(self.frontend(spec))
        
        return x

In [20]:
crnnsa = CRNNSA(main_dict)
print(crnnsa)

CRNNSA(
  (frontend): Frontend_mine(
    (spec_bn): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv_block1): BlockChoi(
      (conv): Conv2dSame(1, 128, kernel_size=(3, 3), stride=(1, 1))
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): ELU(alpha=1.0)
      (pool): MaxPool2d(kernel_size=(3, 1), stride=(3, 1), padding=0, dilation=1, ceil_mode=False)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (conv_block2): BlockChoi(
      (conv): Conv2dSame(128, 128, kernel_size=(3, 3), stride=(1, 1))
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): ELU(alpha=1.0)
      (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (conv_block3): BlockChoi(
      (conv): Conv2dSame(128, 256, kernel_size=(3, 3), stride=(1, 1))
      (bn):

In [4]:
input_length = 5
input_length = int(input_length*16000/256)

In [5]:
spec = torch.FloatTensor(np.load("/import/c4dm-datasets/rmri_self_att/msd/1/1/113801.npy")[np.newaxis,
                                                                                           np.newaxis,
                                                                                           :,
                                                                                           :input_length])
spec.shape

torch.Size([1, 1, 96, 312])

In [21]:
#https://towardsdatascience.com/temporal-convolutional-networks-the-next-revolution-for-time-series-8990af826567
x = nn.Conv1d(256,256, 3)(crnnsa.frontend(spec))
x = nn.Conv1d(256,256, 3, dilation=2)(x)
x = nn.Conv1d(256,256, 3, dilation=4)(x)
x = nn.Conv1d(256,256, 3, dilation=8)(x)
x.shape
              

torch.Size([1, 256, 48])

In [8]:
crnnsa.frontend(spec).shape

torch.Size([1, 256, 312])

In [2]:
dataLoader = get_DataLoader(batch_size=1, input_length=5, mode="valid", num_workers=6)

In [9]:
ctr = 0
for x,y in dataLoader:
    print(x.shape)
    print(y.shape)
    ctr+=1
    if ctr==10:
        break

torch.Size([1, 6, 96, 312])
torch.Size([1, 50])
torch.Size([1, 6, 96, 312])
torch.Size([1, 50])
torch.Size([1, 12, 96, 312])
torch.Size([1, 50])
torch.Size([1, 12, 96, 312])
torch.Size([1, 50])
torch.Size([1, 12, 96, 312])
torch.Size([1, 50])
torch.Size([1, 12, 96, 312])
torch.Size([1, 50])
torch.Size([1, 6, 96, 312])
torch.Size([1, 50])
torch.Size([1, 6, 96, 312])
torch.Size([1, 50])
torch.Size([1, 12, 96, 312])
torch.Size([1, 50])
torch.Size([1, 12, 96, 312])
torch.Size([1, 50])


In [7]:
print(crnnsa.frontend(x).shape)
print(crnnsa(x).shape)

NameError: name 'x' is not defined

In [ ]:
whole_spec = np.load("/import/c4dm-datasets/rmri_self_att/msd/1/1/113801.npy")

n_chunks = whole_spec.shape[1] // input_length
spec = np.zeros((n_chunks,whole_spec.shape[0],input_length))
for i in range(n_chunks):
    spec[i]=whole_spec[:,i*input_length:(i+1)*input_length]

In [ ]:
spec.shape

In [ ]:
whole_spec.shape